In [1]:
!pip install --no-cache-dir package_name optuna
# !pip install --no-cache-dir package_name scikit-optimize


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords, wordnet
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
import os
import string
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
from sklearn.manifold import TSNE
import itertools
import collections
from collections import Counter
from gensim import corpora, models
from gensim.models import Word2Vec, word2vec, LdaModel
import optuna
from hyperopt import hp, tpe, fmin
import warnings
warnings.filterwarnings("ignore")
import gensim
from gensim.utils import simple_preprocess
from datetime import datetime
# import spacy
from nltk import pos_tag
import pickle


In [3]:
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('drive/MyDrive/5.TopicModeling/Data/final.csv', encoding='latin-1')
df['tweet_created_at'] = pd.to_datetime(df["tweet_created_at"])
df['tweet_created_at'] = df['tweet_created_at'].dt.strftime('%Y-%m-%d')
df

,Unnamed: 0,tweet_id,tweet_text,tweet_created_at,tweet_source,tweet_lang,tweet_possibly_sensitive,tweet_retweet_count,tweet_reply_count,tweet_like_count,...,place_name,place_full_name,place_country,place_country_code,place_type,clean_text,tokens,token,lemm_text,lemm_textA
0,0,9.041318e+17,"#WILDFIRE brings smoke, ash, unhealthful air q...",2017-09-02,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,wildfire brings smoke ash unhealthful air qual...,"['wildfire', 'brings', 'smoke', 'ash', 'unheal...","[('brings', 'NNS'), ('smoke', 'VBD'), ('ash', ...","['brings', 'smoke', 'ash', 'unhealthful', 'air...","['brings', 'smoke', 'ash', 'air', 'quality', '..."
1,1,9.041317e+17,"#WILDFIRE brings smoke, ash, unhealthful air q...",2017-09-02,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,wildfire brings smoke ash unhealthful air qual...,"['wildfire', 'brings', 'smoke', 'ash', 'unheal...","[('brings', 'NNS'), ('smoke', 'VBD'), ('ash', ...","['brings', 'smoke', 'ash', 'unhealthful', 'air...","['brings', 'smoke', 'ash', 'air', 'quality', '..."
2,2,9.041309e+17,California on fire and Texas under water smh,2017-09-02,NaN,en,True,0.0,0.0,13.0,...,NaN,NaN,NaN,NaN,NaN,california fire texas water smh,"['california', 'texas', 'water', 'smh']","[('california', 'NN'), ('fire', 'NN'), ('texas...","['california', 'fire', 'texas', 'water', 'smh']","['california', 'fire', 'texas', 'water', 'smh']"
3,3,9.041304e+17,MVC: 6000 block of Tom Higgins Lane. Lake Shan...,2017-09-02,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,mvc 6000 block tom higgins lane lake shannon b...,"['mvc', 'block', 'tom', 'higgins', 'lane', 'la...","[('mvc', 'NN'), ('6000', 'CD'), ('block', 'NN'...","['mvc', 'block', 'tom', 'higgins', 'lane', 'la...","['block', 'tom', 'lane', 'lake', 'boat', 'laun..."
4,4,9.041302e+17,"Wildlife brings smoke, ash, unhealthful air qu...",2017-09-02,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,wildlife brings smoke ash unhealthful air qual...,"['wildlife', 'brings', 'smoke', 'ash', 'unheal...","[('wildlife', 'JJ'), ('brings', 'NNS'), ('smok...","['wildlife', 'brings', 'smoke', 'ash', 'unheal...","['wildlife', 'brings', 'smoke', 'ash', 'air', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301471,301471,9.415234e+17,"Pasadena, California â¢ \nJet Propulsion Labo...",2017-12-15,NaN,en,False,3.0,0.0,9.0,...,Mid-Wilshire,"Mid-Wilshire, Los Angeles",Verenigde Staten,US,neighborhood,pasadena california jet propulsion laboratory ...,"['pasadena', 'california', 'jet', 'propulsion'...","[('pasadena', 'NN'), ('california', 'NN'), ('j...","['pasadena', 'california', 'jet', 'propulsion'...","['california', 'jet', 'fire', 'personnel', 'as..."
301472,301472,9.419101e+17,So sad.........:(\nPlease help the family of t...,2017-12-16,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,sad please help family california firefighter ...,"['sad', 'help', 'family', 'california', 'firef...","[('sad', 'JJ'), ('please', 'NN'), ('help', 'VB...","['sad', 'please', 'help', 'family', 'californi...","['sad', 'please', 'help', 'family', 'californi..."
301473,301473,9.424684e+17,My heart goes out to all those affected by the...,2017-12-17,NaN,en,True,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,heart goes affected california wildfires sad,"['heart', 'goes', 'affected', 'california', 'w...","[('heart', 'NN'), ('goes', 'VBZ'), ('affected'...","['heart', 'go', 'affect', 'california', 'wildf...","['heart', 'go', 'affect', 'california', 'wildf..."
301474,301474,9.458634e+17,@realDonaldTrump Thank you Mr Trump for your i...,2017-12-27,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,thank mr trump incredible leadership divine sk...,"['thank', 'mr', 'trump', 'incredible', 'leader...","[('thank', 'JJ'), ('trump', 'NN'), ('incredibl...","['thank', 'trump', 'incredible', 'leadership',...","['thank', 'trump', 'incredible', 'leadership',..."


### separating data into three difference period

09/08/2017-10/07/2017: Before wildfire

10/08/2017-10/31/2017: During wildfire

11/01/2017-12/01/2017: After wildfire

In [6]:
#before wildfire
start_date = '2017-09-08'
end_date = '2017-10-07'

filtered_df = df[(df['tweet_created_at'] >= start_date) & (df['tweet_created_at'] <= end_date)]
len(filtered_df)
filtered_df

,Unnamed: 0,tweet_id,tweet_text,tweet_created_at,tweet_source,tweet_lang,tweet_possibly_sensitive,tweet_retweet_count,tweet_reply_count,tweet_like_count,...,place_name,place_full_name,place_country,place_country_code,place_type,clean_text,tokens,token,lemm_text,lemm_textA
5545,5545,9.063062e+17,#FraserValley #AirQuality index back to normal...,2017-09-08,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,fraser valley air quality index back normal en...,"['fraser', 'valley', 'air', 'quality', 'index'...","[('index', 'NN'), ('back', 'RB'), ('normal', '...","['index', 'back', 'normal', 'enjoy', 'weekend']","['back', 'normal', 'enjoy', 'weekend']"
5546,5546,9.063059e+17,@HarveyLevinTMZ Harvey unless u repent &amp; t...,2017-09-08,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,harvey unless u repent amp turn away ur sins u...,"['harvey', 'repent', 'amp', 'turn', 'away', 'u...","[('harvey', 'NN'), ('unless', 'IN'), ('repent'...","['harvey', 'repent', 'amp', 'turn', 'away', 's...","['harvey', 'repent', 'amp', 'turn', 'away', 's..."
5547,5547,9.063022e+17,@tessgarman A bunch of water PokÃ©mon playing ...,2017-09-08,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,bunch water pokmon playing lake lot fire types...,"['bunch', 'water', 'pokmon', 'playing', 'lake'...","[('bunch', 'JJ'), ('water', 'NN'), ('pok', 'NN...","['bunch', 'water', 'pok', 'mon', 'play', 'lake...","['bunch', 'water', 'mon', 'play', 'lake', 'lot..."
5548,5548,9.063012e+17,you are not guaranteed tomorrow\nbelieve in Je...,2017-09-08,NaN,en,True,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,guaranteed tomorrow believe jesus death catche...,"['guaranteed', 'tomorrow', 'believe', 'jesus',...","[('guaranteed', 'VBN'), ('tomorrow', 'NN'), ('...","['guarantee', 'tomorrow', 'believe', 'jesus', ...","['guarantee', 'tomorrow', 'believe', 'jesus', ..."
5549,5549,9.063005e+17,"Canada, Montana, Oregon, &amp; California are ...",2017-09-08,NaN,en,True,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,canada montana oregon amp california fire texa...,"['canada', 'montana', 'oregon', 'amp', 'califo...","[('canada', 'NN'), ('montana', 'NN'), ('oregon...","['canada', 'montana', 'oregon', 'amp', 'califo...","['canada', 'montana', 'oregon', 'amp', 'califo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300722,300722,9.154065e+17,@realDonaldTrump A miracle wouldâve been the...,2017-10-04,NaN,en,False,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,miracle wouldve cop got single shot fired ur m...,"['miracle', 'wouldve', 'cop', 'got', 'single',...","[('miracle', 'NN'), ('would', 'MD'), ('cop', '...","['miracle', 'cop', 'get', 'single', 'shot', 'f...","['miracle', 'cop', 'get', 'single', 'shot', 'f..."
300723,300723,9.153460e+17,Why is @ABC @CBSNews @FoxNews @cnnbrk still sh...,2017-10-03,NaN,en,False,6.0,5.0,15.0,...,NaN,NaN,NaN,NaN,NaN,still showing actual shooting complete gunfire...,"['showing', 'actual', 'shooting', 'complete', ...","[('still', 'RB'), ('showing', 'VBG'), ('actual...","['still', 'show', 'actual', 'shoot', 'complete...","['still', 'show', 'actual', 'shoot', 'complete..."
300724,300724,9.152799e+17,@Futhead Are them bringing back guest mode!!! ...,2017-10-03,NaN,en,False,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,bringing back guest mode fifa used team play h...,"['bringing', 'guest', 'mode', 'fifa', 'team', ...","[('bringing', 'VBG'), ('back', 'RB'), ('guest'...","['bring', 'back', 'guest', 'mode', 'fifa', 'us...","['bring', 'back', 'guest', 'use', 'team', 'pla..."
300725,300725,9.152529e+17,Poor herman munster look alike ...lol sad old ...,2017-10-03,NaN,en,False,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,poor herman munster look alike lol sad old sou...,"['poor', 'herman', 'munster', 'look', 'alike',...","[('poor', 'JJ'), ('herman', 'NN'), ('munster',...","['poor', 'herman', 'munster', 'look', 'lol', '...","['poor', 'look', 'lol', 'sad', 'old', 'soul', ..."


# Word Distribution

In [7]:
wordcounter = Counter()
# Update the Counter with all tokens in the 'lemm_text' column
for tokens in filtered_df['lemm_text']:
    wordcounter.update(tokens)
filtered_df['lemm_textA'] = [[word for word in tokens if wordcounter[word] >= 100] for tokens in filtered_df['lemm_text']]
filtered_df

,Unnamed: 0,tweet_id,tweet_text,tweet_created_at,tweet_source,tweet_lang,tweet_possibly_sensitive,tweet_retweet_count,tweet_reply_count,tweet_like_count,...,place_name,place_full_name,place_country,place_country_code,place_type,clean_text,tokens,token,lemm_text,lemm_textA
5545,5545,9.063062e+17,#FraserValley #AirQuality index back to normal...,2017-09-08,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,fraser valley air quality index back normal en...,"['fraser', 'valley', 'air', 'quality', 'index'...","[('index', 'NN'), ('back', 'RB'), ('normal', '...","['index', 'back', 'normal', 'enjoy', 'weekend']","[[, ', i, n, d, e, x, ', ,, , ', b, a, c, k, ..."
5546,5546,9.063059e+17,@HarveyLevinTMZ Harvey unless u repent &amp; t...,2017-09-08,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,harvey unless u repent amp turn away ur sins u...,"['harvey', 'repent', 'amp', 'turn', 'away', 'u...","[('harvey', 'NN'), ('unless', 'IN'), ('repent'...","['harvey', 'repent', 'amp', 'turn', 'away', 's...","[[, ', h, a, r, v, e, y, ', ,, , ', r, e, p, ..."
5547,5547,9.063022e+17,@tessgarman A bunch of water PokÃ©mon playing ...,2017-09-08,NaN,en,True,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,bunch water pokmon playing lake lot fire types...,"['bunch', 'water', 'pokmon', 'playing', 'lake'...","[('bunch', 'JJ'), ('water', 'NN'), ('pok', 'NN...","['bunch', 'water', 'pok', 'mon', 'play', 'lake...","[[, ', b, u, n, c, h, ', ,, , ', w, a, t, e, ..."
5548,5548,9.063012e+17,you are not guaranteed tomorrow\nbelieve in Je...,2017-09-08,NaN,en,True,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,guaranteed tomorrow believe jesus death catche...,"['guaranteed', 'tomorrow', 'believe', 'jesus',...","[('guaranteed', 'VBN'), ('tomorrow', 'NN'), ('...","['guarantee', 'tomorrow', 'believe', 'jesus', ...","[[, ', g, u, a, r, a, n, t, e, e, ', ,, , ', ..."
5549,5549,9.063005e+17,"Canada, Montana, Oregon, &amp; California are ...",2017-09-08,NaN,en,True,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,canada montana oregon amp california fire texa...,"['canada', 'montana', 'oregon', 'amp', 'califo...","[('canada', 'NN'), ('montana', 'NN'), ('oregon...","['canada', 'montana', 'oregon', 'amp', 'califo...","[[, ', c, a, n, a, d, a, ', ,, , ', m, o, n, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300722,300722,9.154065e+17,@realDonaldTrump A miracle wouldâve been the...,2017-10-04,NaN,en,False,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,miracle wouldve cop got single shot fired ur m...,"['miracle', 'wouldve', 'cop', 'got', 'single',...","[('miracle', 'NN'), ('would', 'MD'), ('cop', '...","['miracle', 'cop', 'get', 'single', 'shot', 'f...","[[, ', m, i, r, a, c, l, e, ', ,, , ', c, o, ..."
300723,300723,9.153460e+17,Why is @ABC @CBSNews @FoxNews @cnnbrk still sh...,2017-10-03,NaN,en,False,6.0,5.0,15.0,...,NaN,NaN,NaN,NaN,NaN,still showing actual shooting complete gunfire...,"['showing', 'actual', 'shooting', 'complete', ...","[('still', 'RB'), ('showing', 'VBG'), ('actual...","['still', 'show', 'actual', 'shoot', 'complete...","[[, ', s, t, i, l, l, ', ,, , ', s, h, o, w, ..."
300724,300724,9.152799e+17,@Futhead Are them bringing back guest mode!!! ...,2017-10-03,NaN,en,False,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,bringing back guest mode fifa used team play h...,"['bringing', 'guest', 'mode', 'fifa', 'team', ...","[('bringing', 'VBG'), ('back', 'RB'), ('guest'...","['bring', 'back', 'guest', 'mode', 'fifa', 'us...","[[, ', b, r, i, n, g, ', ,, , ', b, a, c, k, ..."
300725,300725,9.152529e+17,Poor herman munster look alike ...lol sad old ...,2017-10-03,NaN,en,False,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,poor herman munster look alike lol sad old sou...,"['poor', 'herman', 'munster', 'look', 'alike',...","[('poor', 'JJ'), ('herman', 'NN'), ('munster',...","['poor', 'herman', 'munster', 'look', 'lol', '...","[[, ', p, o, o, r, ', ,, , ', h, e, r, m, a, ..."


# Find dimension

In [9]:
# Create an instance of CountVectorizer
vectorizer = CountVectorizer(
                             max_features = 1000)

# document_texts = [' '.join(tokens) for tokens in df['lemm_text']]
document_texts = [tokens for tokens in filtered_df['lemm_text']]


# Fit the vectorizer on the 'lemm_text' column
vectorizer.fit(document_texts)

# Transform the 'lemm_text' column into a count vector representation
count_vector = vectorizer.transform(document_texts)

# Get the dimensions of the count vector
vector_dimensions = count_vector.shape

# Print the dimensions
print("Count Vector Dimensions:", vector_dimensions)


Count Vector Dimensions: (29733, 1000)


In [ ]:
# # Print the top words for each topic

# ldask10 = LatentDirichletAllocation(n_components = 10, random_state = 5566)
# ldask10.fit(X)

# feature_names = vectorizer.get_feature_names_out()
# for topic_idx, topic in enumerate(ldask10.components_):
#     print(f"Topic #{topic_idx + 1}:")
#     top_words_idx = topic.argsort()[:-16:-1]  # Select top 15 words
#     top_words = [feature_names[i] for i in top_words_idx]
#     print(top_words)
#     print()

In [ ]:
# # Define the objective function for optimization
# def objective(trial):
#     n_topics = trial.suggest_int('n_topics', 10, 100)
#     alpha = trial.suggest_uniform('alpha', 0.1, 1.0)
#     beta = trial.suggest_uniform('beta', 0.1, 1.0)

#     lda = LatentDirichletAllocation(
#         n_components=n_topics,
#         doc_topic_prior=alpha,
#         topic_word_prior=beta,
#         random_state=42
#     )

#     lda.fit(X)
#     perplexity = lda.perplexity(X)
#     return perplexity

# # Run the optimization
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=50)

# # Print the best hyperparameters and objective value
# best_params = study.best_params
# best_value = study.best_value
# print("Best Hyperparameters: ", best_params)
# print("Best Objective Value: ", best_value)

# Gensim LDA

In [7]:
# Create a list of lists from the preprocessed documents
# processed_docs = [doc.split() for doc in df['lemm_text']]
processed_docs = [re.sub(r'[\[\],]', '', doc).split() for doc in filtered_df['lemm_text']]
dictionary = corpora.Dictionary(processed_docs)
# dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)  # Limit the dictionary size
# Convert the dictionary into a bag-of-words representation
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
# Initialize the LDA model with 10 top
num_topics = 10
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, random_state = 5566)

In [11]:
# Increase the number of topics from 10 to 100
new_num_topics = 100

while num_topics < new_num_topics:
    # Increase the number of topics
    num_topics += 10

    # Re-train the LDA model with the updated number of topics
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary)

    # Evaluate the model's performance (using coherence score as an example)
    coherence_score = models.CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v').get_coherence()

    print(f"Number of topics: {num_topics}, Coherence Score: {coherence_score}")

    # Repeat the process until the desired number of topics or satisfactory results are achieved

Number of topics: 20, Coherence Score: 0.36231962829980857
Number of topics: 30, Coherence Score: 0.3646243974585282
Number of topics: 40, Coherence Score: 0.4181794061075026
Number of topics: 50, Coherence Score: 0.44822139994496746
Number of topics: 60, Coherence Score: 0.46063849009507246
Number of topics: 70, Coherence Score: 0.4688026652940541
Number of topics: 80, Coherence Score: 0.4489412176992598
Number of topics: 90, Coherence Score: 0.43671210371900554
Number of topics: 100, Coherence Score: 0.40218245486648174


In [12]:
#try new topic model
lda_modelA = models.LdaModel(corpus, num_topics=70, id2word=dictionary)
for i,topic in lda_modelA.show_topics(formatted=True, num_topics=70, num_words=15):
    print(str(i)+": "+ topic)
    print()

0: 0.185*"'time'" + 0.100*"'fire'" + 0.093*"'well'" + 0.080*"'home'" + 0.059*"'lose'" + 0.053*"'little'" + 0.048*"'smoke'" + 0.045*"'get'" + 0.042*"'lot'" + 0.037*"'smell'" + 0.036*"'sound'" + 0.027*"'hard'" + 0.025*"'hold'" + 0.023*"'tho'" + 0.019*"'suck'"

1: 0.318*"'need'" + 0.104*"'movie'" + 0.099*"'fire'" + 0.053*"'cry'" + 0.044*"'problem'" + 0.029*"'loss'" + 0.026*"'smoking'" + 0.025*"'victim'" + 0.023*"'dragon'" + 0.018*"'smoke'" + 0.014*"'festival'" + 0.014*"'strong'" + 0.013*"'horrible'" + 0.012*"'shake'" + 0.010*"'rap'"

2: 0.121*"'police'" + 0.080*"'lmao'" + 0.070*"'fire'" + 0.052*"'drive'" + 0.051*"'wrong'" + 0.044*"'officer'" + 0.039*"'firework'" + 0.037*"'office'" + 0.037*"'entire'" + 0.030*"'fighter'" + 0.027*"'app'" + 0.026*"'leader'" + 0.024*"'get'" + 0.019*"'smoke'" + 0.018*"'student'"

3: 0.151*"'first'" + 0.129*"'stop'" + 0.086*"'week'" + 0.073*"'fire'" + 0.068*"'drill'" + 0.052*"'city'" + 0.049*"'spot'" + 0.046*"'least'" + 0.045*"'smoke'" + 0.032*"'cover'" + 0.029*

#Parameter Modification (Optuna)

In [13]:
def objective(trial):
    num_topics = trial.suggest_int('num_topics', 10, 100)
    # iterations = trial.suggest_int('iterations', 50, 100)
    alpha = trial.suggest_categorical('alpha', ['symmetric', 'asymmetric'])
    eta = trial.suggest_float('eta', 0.1, 1.0)
    # rounded_eta = round(suggested_eta, 2)

    # Handle None values for hyperparameters
    # alpha = 1.0 if alpha is None else alpha
    # beta = 1.0 if beta is None else beta

    # Train the LDA model with the current hyperparameters
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics,
                         alpha=alpha, eta=eta)

    # Calculate coherence score for the model
    coherence_model = models.CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    return coherence_score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Print the best hyperparameters and coherence score
best_params = study.best_params
best_coherence = study.best_value
print("Best number of topics:", best_params['num_topics'])
print("Best alpha:", best_params['alpha'])
print("Best beta:", best_params['eta'])
print("Best coherence score:", best_coherence)


[I 2023-06-29 04:31:59,089] A new study created in memory with name: no-name-8c55a914-74f0-4441-ab51-e57f9f90a6d7
[I 2023-06-29 04:32:08,815] Trial 0 finished with value: 0.39862774110347227 and parameters: {'num_topics': 25, 'alpha': 'symmetric', 'eta': 0.3625885127687881}. Best is trial 0 with value: 0.39862774110347227.
[I 2023-06-29 04:32:21,705] Trial 1 finished with value: 0.42682397810315764 and parameters: {'num_topics': 66, 'alpha': 'symmetric', 'eta': 0.6523651667942126}. Best is trial 1 with value: 0.42682397810315764.
[I 2023-06-29 04:32:33,185] Trial 2 finished with value: 0.3798391168247842 and parameters: {'num_topics': 52, 'alpha': 'symmetric', 'eta': 0.9110253877223508}. Best is trial 1 with value: 0.42682397810315764.
[I 2023-06-29 04:32:42,489] Trial 3 finished with value: 0.45011957375032846 and parameters: {'num_topics': 22, 'alpha': 'symmetric', 'eta': 0.6318462068937759}. Best is trial 3 with value: 0.45011957375032846.
[I 2023-06-29 04:32:52,540] Trial 4 finishe

Best number of topics: 26
Best alpha: asymmetric
Best beta: 0.47724985983520574
Best coherence score: 0.5473497579978244


In [ ]:
modified_lda_model = LdaModel(corpus, id2word = dictionary,
                       num_topics = best_params['num_topics'], alpha = best_params['alpha'],
                       eta = best_params['eta'])
for i,topic in modified_lda_model.show_topics(formatted=True, num_topics=best_params['num_topics'], num_words=15):
    print(str(i)+": "+ topic)
    print()

In [15]:
 # save model
# with open('drive/MyDrive/5.TopicModeling/Model/lda_beforeWildfire_mod', 'wb') as file:
    # pickle.dump(modified_lda_model, file)

In [8]:
# Load the LDA model
model_path = 'drive/MyDrive/5.TopicModeling/Model/lda_beforeWildfire_mod'
with open(model_path, 'rb') as file:
    modified_lda_model = pickle.load(file)

#implement model
for i,topic in modified_lda_model.show_topics(formatted=True, num_topics= 26, num_words=15):
    print(str(i)+": "+ topic)
    print()

0: 0.080*"'fire'" + 0.057*"'smoke'" + 0.039*"'sad'" + 0.023*"'get'" + 0.022*"'want'" + 0.015*"'worry'" + 0.011*"'amp'" + 0.011*"'give'" + 0.009*"'tired'" + 0.008*"'make'" + 0.007*"'need'" + 0.007*"'sleep'" + 0.007*"'say'" + 0.007*"'know'" + 0.006*"'time'"

1: 0.089*"'fire'" + 0.015*"'sad'" + 0.012*"'kill'" + 0.010*"'want'" + 0.007*"'kid'" + 0.007*"'player'" + 0.007*"'people'" + 0.007*"'worry'" + 0.007*"'fan'" + 0.006*"'dead'" + 0.006*"'care'" + 0.006*"'police'" + 0.006*"'tonight'" + 0.006*"'beat'" + 0.005*"'game'"

2: 0.115*"'fire'" + 0.050*"'terrible'" + 0.042*"'tire'" + 0.024*"'angry'" + 0.021*"'people'" + 0.016*"'set'" + 0.016*"'sad'" + 0.013*"'want'" + 0.010*"'gun'" + 0.008*"'take'" + 0.008*"'world'" + 0.007*"'go'" + 0.007*"'make'" + 0.007*"'alarm'" + 0.007*"'get'"

3: 0.021*"'attack'" + 0.020*"'find'" + 0.013*"'fire'" + 0.012*"'use'" + 0.012*"'like'" + 0.011*"'name'" + 0.008*"'air'" + 0.008*"'service'" + 0.008*"'rapid'" + 0.007*"'issue'" + 0.007*"'flag'" + 0.006*"'book'" + 0.006*"

# Topic Definition

### Topic 3: Finding and Responding to Threats

The keywords such as "attack," "find," "fire," "use," and "like" suggest a focus on identifying and addressing potential threats or attacks. Other keywords like "name," "air," "service," and "rapid" indicate a possible connection to security services or quick response to security-related issues. Overall, these keywords point towards a topic related to security, investigation, and dealing with potential threats.

### Topic 5: Concerns about a Dumpster Incident or Emergency

The prominent keywords such as "'dumpster'," "'death'," "'fire'," "'flood'," and "'caught'" indicate a potential incident or emergency situation. Other keywords like "'ready'," "'moment'," and "'act'" suggest a sense of urgency or preparedness in response to the situation. The mention of "'president'" may indicate a political context or reference, but without further context, it is unclear how it relates to the overall topic.

### Topic 7: Socializing and Emotional Experiences

The keywords "drink," "crowd," and "alone" suggest aspects related to social interactions and being in the company of others. The presence of words like "hot," "fire," and "smoke" could indicate a metaphorical or literal reference to intense or challenging situations. Additionally, keywords like "fear" and "best" suggest emotional experiences and personal evaluations.

### Topic 10: Anxiety and Self-Image

The most prominent keyword is "anxiety," indicating that the topic revolves around anxiety. The presence of words like "face" and "fix" suggests a connection to self-image or appearance. Other keywords such as "doubt," "date," and "trust" could be related to social interactions or relationships impacted by anxiety. The keywords "pill" and "CBD" hint at potential methods for managing anxiety symptoms. Overall, this topic appears to focus on anxiety in relation to self-image, social interactions, and potential strategies for coping.

### Topic 11: Smoking and its Effects

The keywords such as "'smoke'," "'cigarette'," and "'hand'" indicate a focus on smoking-related elements. Additionally, the presence of "'day'" suggests a temporal aspect, possibly referring to the frequency or duration of smoking. The mention of "'depression'" also implies a potential connection between smoking and mental health. Overall, the keywords indicate a discussion or concern related to smoking and its impact, potentially including aspects such as addiction, health effects, and associated behaviors.

### Topic 14: Negative Expressions and Social/Political Discourse

Based on the keywords provided, the topic for this list appears to involve negative language and potentially contentious or derogatory terms. Some of the notable keywords include "'sick'," "'bitch'," "'tell'," "'feeling'," and "'fuck'." These terms indicate a negative or aggressive tone. Additionally, terms like "'solitude'," "'female'," "'Obama'," "'political'," and "'amendment'" suggest potential connections to social or political discussions.

### Topic 16: Boredom, People, and Racist Remarks

The words "bore," "ppl" (short for people), and "racist" are the most prominent keywords in the list. This suggests a discussion or expression of boredom, interactions with people, and references to racist remarks. Other keywords such as "stupid," "tax," and "energy" indicate additional aspects of the topic. However, the primary focus appears to be on boredom and social interactions with people, particularly concerning racist comments.

### Topic 17: Horror Movies and Entertainment

The presence of words like "'horror'," "'movie'," "'fun'," and references to entertainment platforms such as "'xbox'," "'amazon'," "'apple'," and "'roku'" suggests a discussion or interest in horror movies and related entertainment options. The additional keywords like "'wood'," "'tree'," "'fog'," and "'footage'" may indicate elements commonly associated with horror movies, further reinforcing the topic label.

### Topic 18: Symbols and Identity

The presence of words like "'emblem,'" "'warrior,'" "'terrorist,'" "'race,'" and "'hero'" suggests a discussion related to symbols, identities, and their association with conflict or war. The keywords "'stand,'" "'press,'" "'democrat,'" and "'popular'" might indicate a connection to social or political issues. Overall, these keywords convey a theme of symbols, identities, and their significance in the context of conflict or societal divisions.



